# K-means Clustering  (การคลัสเตอร์ K-means)

ในแบบฝึกหัดนี้ คุณจะได้เรียนรู้การใช้งานอัลกอริทึม K-means และนำไปใช้ในการบีบอัดภาพ

* คุณจะเริ่มต้นด้วยชุดข้อมูลตัวอย่างที่จะช่วยให้คุณเข้าใจวิธีการทำงานของอัลกอริทึม K-means
* หลังจากนั้น คุณจะใช้อัลกอริทึม K-means เพื่อบีบอัดภาพ โดยลดจำนวนสีที่เกิดขึ้นในภาพให้เหลือเพียงสีที่พบบ่อยที่สุดในภาพนั้น




# โครงร่าง
- [ 1 - การใช้งาน K-means](#1)
  - [ 1.1 การค้นหาเซนทรอยด์ที่ใกล้ที่สุด](#1.1)
    - [ แบบฝึกหัด 1](#ex01)
  - [ 1.2 การคำนวณค่าเฉลี่ยของเซนทรอยด์](#1.2)
    - [ แบบฝึกหัด 2](#ex02)
- [ 2 - K-means บนชุดข้อมูลตัวอย่าง ](#2)
- [ 3 - การเริ่มต้นแบบสุ่ม](#3)
- [ 4 - การบีบอัดภาพด้วย K-means](#4)
  - [ 4.1 ชุดข้อมูล](#4.1)
  - [ 4.2 K-Means บนพิกเซลของภาพ](#4.2)
  - [ 4.3 บีบอัดภาพ](#4.3)


**หมายเหตุ:** เพื่อป้องกันข้อผิดพลาดจากตัวตรวจการบ้าน คุณไม่ได้รับอนุญาตให้แก้ไขหรือลบคเซลล์ที่ไม่ได้ให้คะแนน โปรดหลีกเลี่ยงการเพิ่มเซลล์ใหม่ด้วย 

**เมื่อคุณผ่านการมอบหมายงานนี้แล้ว** และต้องการทดลองกับโค้ดที่ไม่ได้ให้คะแนน คุณสามารถทำตามคำแนะนำที่ด้านล่างของโน้ตบุ๊กนี้

ก่อนอื่น ให้รันเซลล์ด้านล่างเพื่อติดตั้งแพ็คเกจที่จำเป็นสำหรับการบ้านนี้:
- [numpy](https://numpy.org/) เป็นแพ็คเกจพื้นฐานสำหรับการคำนวณทางวิทยาศาสตร์ด้วย Python
- [matplotlib](http://matplotlib.org) เป็นไลบรารีที่นิยมใช้ในการวาดกราฟใน Python
- `utils.py` ประกอบด้วยฟังก์ชันช่วยเหลือสำหรับการบ้านนี้ คุณไม่จำเป็นต้องแก้ไขโค้ดในไฟล์นี้

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *

%matplotlib inline

<a name="1"></a>
## 1 - อัลกอริทึม K-means

อัลกอริทึม K-means เป็นวิธีการจัดกลุ่มข้อมูลที่คล้ายคลึงกันโดยอัตโนมัติ 

* โดยเฉพาะอย่างยิ่ง คุณจะได้รับชุดฝึก  $\{x^{(1)}, ..., x^{(m)}\}$,และคุณต้องการจัดกลุ่มข้อมูลเป็น "กลุ่ม" ที่เชื่อมโยงกันไม่กี่กลุ่ม


* K-means เป็นขั้นตอนแบบวนซ้ำที่
     * เริ่มต้นด้วยการเดาเซนทรอยด์เริ่มต้น และจากนั้น
     * ปรับปรุงการเดานี้โดย 
         * กำหนดตัวอย่างให้กับเซนทรอยด์ที่ใกล้ที่สุดซ้ำๆ และจากนั้น
         * คำนวณเซนทรอยด์ใหม่โดยยึดตามการกำหนด

* ในรหัสเทียม อัลกอริทึม K-means มีดังนี้:

    ``` python
    # Initialize centroids
    # K is the number of clusters
    centroids = kMeans_init_centroids(X, K)
    
    for iter in range(iterations):
        # Cluster assignment step: 
        # Assign each data point to the closest centroid. 
        # idx[i] corresponds to the index of the centroid 
        # assigned to example i
        idx = find_closest_centroids(X, centroids)

        # Move centroid step: 
        # Compute means based on centroid assignments
        centroids = compute_centroids(X, idx, K)
    ```


* ลูปภายในของอัลกอริทึมจะทำสองขั้นตอนซ้ำๆ:
    1. กำหนดแต่ละตัวอย่างการฝึก  $x^{(i)}$  ให้กับเซนทรอยด์ที่ใกล้ที่สุด และ
    2. คำนวณค่าเฉลี่ยของแต่ละเซนทรอยด์โดยใช้จุดที่ถูกกำหนดให้
    
* อัลกอริทึม  $K$-จะลู่เข้าสู่ชุดค่าเฉลี่ยสุดท้ายของเซนทรอยด์เสมอ

* อย่างไรก็ตาม โซลูชันที่ลู่เข้าอาจไม่เหมาะสมเสมอไป และขึ้นอยู่กับการตั้งค่าเริ่มต้นของเซนทรอยด์
    * ดังนั้น ในทางปฏิบัติ อัลกอริทึม K-means มักจะถูกเรียกใช้หลายครั้งด้วยการเริ่มต้นแบบสุ่มที่แตกต่างกัน
    * วิธีหนึ่งในการเลือกจากโซลูชันต่างๆ เหล่านี้จากการเริ่มต้นแบบสุ่มที่แตกต่างกันคือการเลือกโซลูชันที่มีค่าฟังก์ชันต้นทุนต่ำสุด (distortion)


คุณจะนำไปใช้สองเฟสของอัลกอริทึม K-means แยกกัน
ในส่วนถัดไป   
* คุณจะเริ่มต้นด้วยการเติมเต็ม `find_closest_centroid` และจากนั้นดำเนินการเติมเต็ม  `compute_centroids`.

<a name="1.1"></a>
### 1.1 การค้นหาเซนทรอยด์ที่ใกล้ที่สุด

ในเฟส "การกำหนดกลุ่ม" ของอัลกอริทึม K-means อัลกอริทึมจะกำหนดทุกตัวอย่างการฝึก  $x^{(i)}$ ให้กับเซนทรอยด์ที่ใกล้ที่สุด โดยพิจารณาจากตำแหน่งปัจจุบันของเซนทรอยด์

<a name="ex01"></a>
### แบบฝึกหัด 1

งานของคุณคือการเติมโค้ดใน `find_closest_centroids`. 
* ฟังก์ชันนี้รับเมทริกซ์ข้อมูล `X` และตำแหน่งของเซนทรอยด์ทั้งหมดภายใน `centroids` 
* ควรส่งออกอาร์เรย์หนึ่งมิติ `idx` (ซึ่งมีจำนวนองค์ประกอบเท่ากับ `X`) t ที่เก็บดัชนีของเซนทรอยด์ที่ใกล้ที่สุด (ค่าใน $\{0,...,K-1\}$ โดยที่ $K$ คือจำนวนเซนทรอยด์ทั้งหมด) สำหรับทุกตัวอย่างการฝึก *(หมายเหตุ: ช่วงดัชนี 0 ถึง K-1 แตกต่างจากที่แสดงในบทเรียนเล็กน้อย (เช่น 1 ถึง K) เนื่องจากดัชนีรายการ Python เริ่มต้นที่ 0 แทนที่จะเป็น 1)*

* โดยเฉพาะอย่างยิ่ง สำหรับทุกตัวอย่าง $x^{(i)}$  เราตั้งค่า
$$c^{(i)} := j \quad \mathrm{that \; minimizes} \quad ||x^{(i)} - \mu_j||^2,$$
โดยที่
 * $c^{(i)}$ คือดัชนีของเซนทรอยด์ที่ใกล้ที่สุดกับ $x^{(i)}$  (ตรงกับ `idx[i]` ในโค้ดเริ่มต้น) และ
 * $\mu_j$ คือตำแหน่ง (ค่า) ของเซนทรอยด์ที่ $j$’ (เก็บไว้ใน  `centroids` ในโค้ดเริ่มต้น)
 * $||x^{(i)} - \mu_j||$ คือ นอร์ม L2 (L2-norm)
 
หากคุณติดขัด คุณสามารถตรวจสอบคำแนะนำที่นำเสนอหลังเซลล์ด้านล่างเพื่อช่วยคุณในการใช้งาน


In [ ]:
# UNQ_C1
# GRADED FUNCTION: find_closest_centroids

def find_closest_centroids(X, centroids):
    """
    Computes the centroid memberships for every example
    
    Args:
        X (ndarray): (m, n) Input values      
        centroids (ndarray): (K, n) centroids
    
    Returns:
        idx (array_like): (m,) closest centroids
    
    """

    # Set K
    K = centroids.shape[0]

    # You need to return the following variables correctly
    idx = np.zeros(X.shape[0], dtype=int)

    ### START CODE HERE ###
    
        
        
            
            
        
     ### END CODE HERE ###
    
    return idx

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
* ต่อไปนี้เป็นวิธีการโครงสร้างการนำไปใช้โดยรวมสำหรับฟังก์ชันนี้

```python 
def find_closest_centroids(X, centroids):

    # Set K
    K = centroids.shape[0]

    # You need to return the following variables correctly
    idx = np.zeros(X.shape[0], dtype=int)

    ### START CODE HERE ###
    for i in range(X.shape[0]):
        # Array to hold distance between X[i] and each centroids[j]
        distance = [] 
        for j in range(centroids.shape[0]):
            norm_ij = # Your code to calculate the norm between (X[i] - centroids[j])
            distance.append(norm_ij)

        idx[i] = # Your code here to calculate index of minimum value in distance
    ### END CODE HERE ###
    return idx
```

* หากคุณยังติดขัด คุณสามารถตรวจสอบคำใบ้ด้านล่างเพื่อหาวิธีคำนวณ `norm_ij` และ `idx[i]`.
    
    <details>
          <summary><font size="2" color="darkblue"><b>คำแนะนำในการคำนวณ norm_ij</b></font></summary>
           &emsp; &emsp; คุณสามารถใช้ <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html">np.linalg.norm</a> ในการคำนวณ norm 
          <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ norm_ij</b></font></summary>
               &emsp; &emsp; คุณสามารถคำนวณ norm_ij as <code>norm_ij = np.linalg.norm(X[i] - centroids[j]) </code>
           </details>
    </details>

    <details>
          <summary><font size="2" color="darkblue"><b>คำแนะนำในการคำนวณ idx[i]</b></font></summary>
          &emsp; &emsp; คุณสามารถใช้ <a href="https://numpy.org/doc/stable/reference/generated/numpy.argmin.html">np.argmin</a> ในการหาค่า indexที่ต่ำที่สุด 
          <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ idx[i]</b></font></summary>
              &emsp; &emsp; You can compute idx[i] as <code>idx[i] = np.argmin(distance)</code>
          </details>
    </details>
        
    </details>

</details>

    


ตอนนี้มาตรวจสอบการใช้งานของคุณโดยใช้ชุดข้อมูลตัวอย่างกัน

In [ ]:
# Load an example dataset that we will be using
X = load_data()

โค้ดด้านล่างจะพิมพ์ 5 องค์ประกอบแรกในตัวแปร `X` และมิติของตัวแปร

In [ ]:
print("First five elements of X are:\n", X[:5]) 
print('The shape of X is:', X.shape)

In [ ]:
# Select an initial set of centroids (3 Centroids)
initial_centroids = np.array([[3,3], [6,2], [8,5]])

# Find closest centroids using initial_centroids
idx = find_closest_centroids(X, initial_centroids)

# Print closest centroids for the first three elements
print("First three elements in idx are:", idx[:3])

# UNIT TEST
from public_tests import *

find_closest_centroids_test(find_closest_centroids)

**ผลลัพธ์ที่คาดหวัง**:
<table>
  <tr>
    <td> <b>สามองค์ประกอบแรกใน idx คือ
<b></td>
    <td> [0 2 1] </td> 
  </tr>
</table>

<a name="1.2"></a>
### 1.2 การคำนวณค่าเฉลี่ยของเซนทรอยด์

เมื่อกำหนดการกำหนดจุดแต่ละจุดให้กับเซนทรอยด์แล้ว ขั้นตอนที่สองของอัลกอริทึมจะคำนวณค่าเฉลี่ยของจุดที่ถูกกำหนดให้กับเซนทรอยด์แต่ละตัวใหม่


<a name="ex02"></a>
### แบบฝึกหัดที่ 2

โปรดเติมโค้ด  `compute_centroids` ด้านล่างเพื่อคำนวณค่าใหม่สำหรับเซนทรอยด์แต่ละตัว

* โดยเฉพาะอย่างยิ่ง สำหรับเซนทรอยด์  $\mu_k$ แต่ละตัว เราตั้งค่า
$$\mu_k = \frac{1}{|C_k|} \sum_{i \in C_k} x^{(i)}$$ 

    โดยที่ 
    * $C_k$ คือเซตของตัวอย่างที่ถูกกำหนดให้กับเซนทรอยด์ $k$
    * $|C_k|$ คือจำนวนตัวอย่างในเซต $C_k$


* โดยเฉพาะอย่างยิ่ง ถ้าตัวอย่างสองตัว เช่น  $x^{(3)}$ และ  $x^{(5)}$ ถูกกำหนดให้กับเซนทรอยด์  $k=2$,
คุณควรอัปเดต $\mu_2 = \frac{1}{2}(x^{(3)}+x^{(5)})$.

หากคุณติดขัด คุณสามารถตรวจสอบคำใบ้ที่นำเสนอหลังเซลล์ด้านล่างเพื่อช่วยคุณในการใช้งาน

In [ ]:
# UNQ_C2
# GRADED FUNCTION: compute_centroids

def compute_centroids(X, idx, K):
    """
    Returns the new centroids by computing the means of the 
    data points assigned to each centroid.
    
    Args:
        X (ndarray):   (m, n) Data points
        idx (ndarray): (m,) Array containing index of closest centroid for each 
                       example in X. Concretely, idx[i] contains the index of 
                       the centroid closest to example i
        K (int):       number of centroids
    
    Returns:
        centroids (ndarray): (K, n) New centroids computed
    """
    
    # Useful variables
    m, n = X.shape
    
    # You need to return the following variables correctly
    centroids = np.zeros((K, n))
    
    ### START CODE HERE ###
    
        
        
    ### END CODE HERE ## 
    
    return centroids

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
* นี่คือวิธีโครงสร้างการใช้งานโดยรวมสำหรับฟังก์ชันนี้

    ```python 
    def compute_centroids(X, idx, K):
        # Useful variables
        m, n = X.shape
    
        # You need to return the following variables correctly
        centroids = np.zeros((K, n))
    
        ### START CODE HERE ###
        for k in range(K):   
            points = # Your code here to get a list of all data points in X assigned to centroid k  
            centroids[k] = # Your code here to compute the mean of the points assigned
    ### END CODE HERE ## 
    
    return centroids
    ```
  
    หากคุณยังติดขัด คุณสามารถตรวจสอบคำใบ้ด้านล่างเพื่อหาวิธีคำนวณ `points` และ  `centroids[k]`.
    
    <details>
          <summary><font size="2" color="darkblue"><b>คำแนะนำในการคำนวณ points</b></font></summary>
           &emsp; &emsp; สมมติว่าเราต้องการค้นหาค่าทั้งหมดใน X ที่ถูกกำหนดให้กับคลัสเตอร์ <code>k=0</code>. นั่นคือ ค่าที่ตรงกันใน idx สำหรับตัวอย่างเหล่านี้คือ 0 ใน Python เราสามารถทำได้ดังนี้ <code>X[idx == 0]</code>. เช่นเดียวกัน จุดที่กำหนดให้กับเซนทรอยด์ <code>k=1</code> are <code>X[idx == 1]</code>
          <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ points</b></font></summary>
               &emsp; &emsp; คุณสามารถคำนวณ points คือ <code>points = X[idx == k] </code>
           </details>
    </details>

     <details>
          <summary><font size="2" color="darkblue"><b>คำแนะนำในการคำนวณ centroids[k]</b></font></summary>
          &emsp; &emsp; คุณสามารถใช้ <a href="https://numpy.org/doc/stable/reference/generated/numpy.mean.html">np.mean</a> ในการหาค่า mean. โปรดตรวจสอบและตั้งค่าพารามิเตอร์ <code>axis=0</code> 
          <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ centroids[k]</b></font></summary>
              &emsp; &emsp; คุณสามารถคำนวณ centroids[k] คือ <code>centroids[k] = np.mean(points, axis = 0)</code>
          </details>
    </details>
        
    </details>

</details>

    


ตอนนี้ตรวจสอบการใช้งานของคุณโดยการรันเซลด้านล่าง

In [ ]:
K = 3
centroids = compute_centroids(X, idx, K)

print("The centroids are:", centroids)

# UNIT TEST
compute_centroids_test(compute_centroids)

**ผลลัพธ์ที่คาดหวัง**:

2.42830111 3.15792418

5.81350331 2.63365645

7.11938687 3.6166844 

<a name="2"></a>
## 2 - K-means บนชุดข้อมูลตัวอย่าง 

หลังจากที่คุณเสร็จสิ้นฟังก์ชันทั้งสอง (`find_closest_centroids`
และ `compute_centroids`) ขั้นตอนต่อไปคือการรันอัลกอริทึม K-means บนชุดข้อมูล 2D ตัวอย่างเพื่อช่วยให้คุณเข้าใจวิธีการทำงานของ K-means
* เราขอแนะนำให้คุณลองดูฟังก์ชัน  (`run_kMeans`) ด้านล่างเพื่อทำความเข้าใจวิธีการทำงาน 
* สังเกตว่าโค้ดเรียกใช้ฟังก์ชันทั้งสองที่คุณได้นำไปใช้ในลูป

เมื่อคุณรันโค้ดด้านล่าง มันจะสร้างการแสดงภาพที่แสดงขั้นตอนของอัลกอริทึมในแต่ละรอบ
* ในที่สุด รูปของคุณควรจะเหมือนกับรูปที่แสดงในรูปที่ 1
* เซนทรอยด์สุดท้ายคือเครื่องหมาย X สีดำตรงกลางของกลุ่มสี
* คุณสามารถดูวิธีที่เซนทรอยด์เหล่านี้ไปถึงตำแหน่งสุดท้ายได้โดยดูที่เครื่องหมาย X อื่นๆ ที่เชื่อมต่อกับมัน

<img src="images/figure 1.png" width="500" height="500">


**หมายเหตุ**: คุณไม่จำเป็นต้องนำอะไรไปใช้ในส่วนนี้ เพียงแค่รันโค้ดด้านล่าง

In [ ]:
# You do not need to implement anything for this part

def run_kMeans(X, initial_centroids, max_iters=10, plot_progress=False):
    """
    Runs the K-Means algorithm on data matrix X, where each row of X
    is a single example
    """
    
    # Initialize values
    m, n = X.shape
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    previous_centroids = centroids    
    idx = np.zeros(m)
    plt.figure(figsize=(8, 6))

    # Run K-Means
    for i in range(max_iters):
        
        #Output progress
        print("K-Means iteration %d/%d" % (i, max_iters-1))
        
        # For each example in X, assign it to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Optionally plot progress
        if plot_progress:
            plot_progress_kMeans(X, centroids, previous_centroids, idx, K, i)
            previous_centroids = centroids
            
        # Given the memberships, compute new centroids
        centroids = compute_centroids(X, idx, K)
    plt.show() 
    return centroids, idx

In [ ]:
# Load an example dataset
X = load_data()

# Set initial centroids
initial_centroids = np.array([[3,3],[6,2],[8,5]])

# Number of iterations
max_iters = 10

# Run K-Means
centroids, idx = run_kMeans(X, initial_centroids, max_iters, plot_progress=True)

<a name="3"></a>
## 3 - การเริ่มต้นแบบสุ่ม

การกำหนดเซนทรอยด์เริ่มต้นสำหรับชุดข้อมูลตัวอย่างได้รับการออกแบบเพื่อให้คุณเห็นรูปเดียวกับในรูปที่ 1 ในทางปฏิบัติ กลยุทธ์ที่ดีในการเริ่มต้นเซนทรอยด์คือการเลือกตัวอย่างแบบสุ่มจากชุดฝึก

ในส่วนนี้ของแบบฝึกหัด คุณควรทำความเข้าใจวิธีการใช้งานฟังก์ชัน `kMeans_init_centroids` 
* โค้ดจะสุ่มสับเปลี่ยนดัชนีของตัวอย่าง (โดยใช้`np.random.permutation()`). 
* จากนั้น เลือกตัวอย่างแรก $K$ ตัวอย่างตามการสับเปลี่ยนแบบสุ่มของดัชนี
* ซึ่งช่วยให้ตัวอย่างถูกเลือกแบบสุ่มโดยไม่เสี่ยงต่อการเลือกตัวอย่างเดียวกันสองครั้ง

**หมายเหตุ**: คุณไม่จำเป็นต้องใช้งานอะไรสำหรับส่วนนี้ของแบบฝึกหัด

In [ ]:
# You do not need to modify this part

def kMeans_init_centroids(X, K):
    """
    This function initializes K centroids that are to be 
    used in K-Means on the dataset X
    
    Args:
        X (ndarray): Data points 
        K (int):     number of centroids/clusters
    
    Returns:
        centroids (ndarray): Initialized centroids
    """
    
    # Randomly reorder the indices of examples
    randidx = np.random.permutation(X.shape[0])
    
    # Take the first K examples as centroids
    centroids = X[randidx[:K]]
    
    return centroids

คุณสามารถรัน K-Means อีกครั้ง แต่คราวนี้ใช้เซนทรอยด์เริ่มต้นแบบสุ่ม ลองรันเซลล์ด้านล่างหลายๆ ครั้งและสังเกตว่าคลัสเตอร์ที่แตกต่างกันถูกสร้างขึ้นอย่างไร ขึ้นอยู่กับจุดเริ่มต้นที่เลือก

In [ ]:
# Run this cell repeatedly to see different outcomes.

# Set number of centroids and max number of iterations
K = 3
max_iters = 10

# Set initial centroids by picking random examples from the dataset
initial_centroids = kMeans_init_centroids(X, K)

# Run K-Means
centroids, idx = run_kMeans(X, initial_centroids, max_iters, plot_progress=True)

<a name="4"></a>
## 4 - การบีบอัดภาพด้วย K-means

ในแบบฝึกหัดนี้ คุณจะนำ K-means มาใช้ในการบีบอัดภาพ

* ในการแสดงสี 24 บิตแบบตรงไปตรงมาของภาพ$^{2}$,  แต่ละพิกเซลจะถูกแสดงเป็นจำนวนเต็มไร้เครื่องหมาย 8 บิตสามตัว (ตั้งแต่ 0 ถึง 255) ที่ระบุค่าความเข้มของสีแดง สีเขียว และสีน้ำเงิน การเข้ารหัสนี้มักเรียกว่าการเข้ารหัส RGB
* ภาพของเรามีสีหลายพันสี และในส่วนนี้ของแบบฝึกหัด คุณจะลดจำนวนสีลงเหลือ 16 สี
* โดยการลดจำนวนสีลง คุณสามารถแสดงภาพ (บีบอัด) ได้อย่างมีประสิทธิภาพ
* โดยเฉพาะอย่างยิ่ง คุณเพียงแค่ต้องเก็บค่า RGB ของ 16 สีที่เลือก และสำหรับแต่ละพิกเซลในภาพ คุณต้องเก็บเฉพาะดัชนีของสีที่ตำแหน่งนั้น (ซึ่งต้องใช้เพียง 4 บิตเพื่อแสดง 16 ความเป็นไปได้)

ในส่วนนี้ คุณจะใช้อัลกอริทึม K-means เพื่อเลือก 16 สีที่จะใช้แทนภาพที่ถูกบีบอัด

* โดยเฉพาะอย่างยิ่ง คุณจะถือว่าทุกพิกเซลในภาพต้นฉบับเป็นตัวอย่างข้อมูลและใช้อัลกอริทึม K-means เพื่อค้นหา 16 สีที่จัดกลุ่ม (คลัสเตอร์) พิกเซลในพื้นที่ RGB 3 มิติได้ดีที่สุด
* เมื่อคุณคำนวณเซนทรอยด์ของคลัสเตอร์บนภาพแล้ว คุณจะใช้ 16 สีเพื่อแทนที่พิกเซลในภาพต้นฉบับ

<img src="images/figure 2.png" width="500" height="500">

$^{2}$<sub>ภาพที่ใช้ในแบบฝึกหัดนี้เป็นของ Frank Wouters และใช้โดยได้รับอนุญาต.</sub>

<a name="4.1"></a>
### 4.1 ชุดข้อมูล

**โหลดภาพ**

อันดับแรก คุณจะใช้ `matplotlib` เพื่ออ่านภาพต้นฉบับ ดังแสดงด้านล่าง

In [ ]:
# Load an image of a bird
original_img = plt.imread('bird_small.png')

**Visualize image**

คุณสามารถมองเห็นภาพที่เพิ่งโหลดมาโดยใช้โค้ดด้านล่าง

In [ ]:
# Visualizing the image
plt.imshow(original_img)

**ตรวจสอบมิติของตัวแปร**

เช่นเคย คุณจะพิมพ์รูปร่างของตัวแปรของคุณเพื่อทำความคุ้นเคยกับข้อมูลมากขึ้น

In [ ]:
print("Shape of original_img is:", original_img.shape)

ดังที่คุณเห็น เมทริกซ์สามมิติ  `original_img` นี้ถูกสร้างขึ้น โดย 
* ดัชนีสองตัวแรกระบุตำแหน่งพิกเซล
* ดัชนีที่สามแทนสีแดง เขียว หรือน้ำเงิน 

ตัวอย่างเช่น `original_img[50, 33, 2]` ให้ค่าความเข้มสีน้ำเงินของพิกเซลที่แถว 50 และคอลัมน์ 33

#### การประมวลผลข้อมูล

เพื่อเรียกใช้ `run_kMeans`, คุณต้องแปลงเมทริกซ์ `original_img`  ให้เป็นเมทริกซ์สองมิติก่อน

* โค้ดด้านล่างปรับขนาดเมทริกซ์ `original_img` เพื่อสร้างเมทริกซ์สีพิกเซล $m \times 3$ (โดยที่
$m=16384 = 128\times128$)

*หมายเหตุ: Iหากคุณลองแบบฝึกหัดนี้ในภายหลังกับไฟล์ JPG คุณต้องหารค่าพิกเซลด้วย 255 ก่อนเพื่อให้ค่าอยู่ในช่วง 0 ถึง 1 ซึ่งไม่จำเป็นสำหรับไฟล์ PNG (เช่น `bird_small.png`) เนื่องจากโหลดอยู่ในช่วงที่ต้องการแล้ว (ตามที่กล่าวไว้ใน  [plt.imread() documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html)).เราได้แสดงความคิดเห็นไว้ด้านล่างเพื่อให้คุณสามารถยกเลิกการแสดงความคิดเห็นในภายหลังหากคุณต้องการลองใช้ไฟล์อื่น* 

In [ ]:
# Divide by 255 so that all values are in the range 0 - 1 (not needed for PNG files)
# original_img = original_img / 255

# Reshape the image into an m x 3 matrix where m = number of pixels
# (in this case m = 128 x 128 = 16384)
# Each row will contain the Red, Green and Blue pixel values
# This gives us our dataset matrix X_img that we will use K-Means on.

X_img = np.reshape(original_img, (original_img.shape[0] * original_img.shape[1], 3))

<a name="4.2"></a>
### K-Means บนพิกเซลของภาพ
ตอนนี้ ให้รันเซลโค้ดด้านล่างเพื่อเรียกใช้ K-Means บนภาพที่ผ่านการประมวลผลแล้ว

In [ ]:
# Run your K-Means algorithm on this data
# You should try different values of K and max_iters here
K = 16
max_iters = 10

# Using the function you have implemented above. 
initial_centroids = kMeans_init_centroids(X_img, K)

# Run K-Means - this can take a couple of minutes depending on K and max_iters
centroids, idx = run_kMeans(X_img, initial_centroids, max_iters)

In [ ]:
print("Shape of idx:", idx.shape)
print("Closest centroid for the first five elements:", idx[:5])

โค้ดด้านล่างจะพล็อตสีทั้งหมดที่พบในภาพต้นฉบับ ดังที่ได้กล่าวไว้ก่อนหน้านี้ สีของแต่ละพิกเซลจะแสดงด้วยค่า RGB ดังนั้นพล็อตควรมี 3 แกน - R, G และ B คุณจะสังเกตเห็นจุดจำนวนมากด้านล่างแทนสีหลายพันสีในภาพต้นฉบับ เครื่องหมายสีแดงแสดงถึงเซนทรอยด์หลังจากรัน K-means เหล่านี้จะเป็น 16 สีที่จะใช้ในการบีบอัดภาพ

In [ ]:
# Plot the colors of the image and mark the centroids
plot_kMeans_RGB(X_img, centroids, idx, K)

คุณสามารถมองเห็นสีที่แต่ละเครื่องหมายสีแดง (เช่น เซนทรอยด์) ด้านบนด้วยฟังก์ชันด้านล่าง คุณจะเห็นสีเหล่านี้เฉพาะเมื่อคุณสร้างภาพใหม่ในส่วนถัดไป หมายเลขใต้แต่ละสีคือดัชนีของมัน และนี่คือหมายเลขที่คุณเห็นในอาร์เรย์ `idx`

In [ ]:
# Visualize the 16 colors selected
show_centroid_colors(centroids)

<a name="4.3"></a>
### 4.3 Compress the image


หลังจากค้นหาสี $K=16$ สีสูงสุดเพื่อแทนภาพ คุณสามารถกำหนดตำแหน่งพิกเซลแต่ละตำแหน่งให้กับเซนทรอยด์ที่ใกล้ที่สุดโดยใช้ฟังก์ชัน
`find_closest_centroids` 
* นี่ช่วยให้คุณสามารถแทนภาพต้นฉบับโดยใช้การกำหนดเซนทรอยด์ของแต่ละพิกเซล
 
* โปรดทราบว่าคุณได้ลดจำนวนบิตที่จำเป็นในการอธิบายภาพลงอย่างมาก

    * ภาพต้นฉบับต้องใช้ 24 บิต (เช่น 8 บิต x 3 ช่องสัญญาณในการเข้ารหัส RGB) สำหรับแต่ละตำแหน่งพิกเซล  $128\times128$ ซึ่งส่งผลให้ขนาดทั้งหมด $128 \times 128 \times 24 = 393,216$ บิต. 
    * การแสดงผลใหม่ต้องใช้พื้นที่เก็บข้อมูลเพิ่มเติมในรูปแบบของพจนานุกรมสี 16 สี แต่ละสีต้องใช้ 24 บิต แต่ตัวภาพเองต้องการเพียง 4 บิตต่อตำแหน่งพิกเซล
    * ดังนั้นจำนวนบิตสุดท้ายที่ใช้คือ  $16 \times 24 + 128 \times 128 \times 4 = 65,920$ บิต ซึ่งสอดคล้องกับการบีบอัดภาพต้นฉบับประมาณ 6 เท่า

In [ ]:
# Find the closest centroid of each pixel
idx = find_closest_centroids(X_img, centroids)

# Replace each pixel with the color of the closest centroid
X_recovered = centroids[idx, :] 

# Reshape image into proper dimensions
X_recovered = np.reshape(X_recovered, original_img.shape) 

สุดท้าย คุณสามารถดูผลกระทบของการบีบอัดโดยการสร้างภาพใหม่ขึ้นมา โดยอิงจากการกำหนดเซนทรอยด์
* โดยเฉพาะ คุณได้แทนที่แต่ละพิกเซลด้วยค่าของเซนทรอยด์ที่กำหนดให้


* รูปที่ 3 แสดงตัวอย่างการสร้างภาพใหม่ แม้ว่าภาพผลลัพธ์จะยังคงรักษาลักษณะส่วนใหญ่ของภาพต้นฉบับ แต่คุณจะเห็นอาร์ติแฟกต์การบีบอัดบางอย่างเนื่องจากใช้สีน้อยลง



<img src="images/figure 3.png" width="700" height="700">

* รันโค้ดด้านล่างเพื่อดูวิธีการสร้างภาพใหม่โดยใช้ 16 สีที่เลือกไว้ก่อนหน้านี้

In [ ]:
# Display original image
fig, ax = plt.subplots(1,2, figsize=(16,16))
plt.axis('off')

ax[0].imshow(original_img)
ax[0].set_title('Original')
ax[0].set_axis_off()


# Display compressed image
ax[1].imshow(X_recovered)
ax[1].set_title('Compressed with %d colours'%K)
ax[1].set_axis_off()

**ขอแสดงความยินดี!
คุณได้เรียนรู้การใช้ K-means clustering แล้ว ในบทเรียนต่อไป คุณจะได้เรียนรู้เกี่ยวกับกรณีการใช้งานอีกอย่างหนึ่งของการเรียนรู้แบบไม่มีผู้สอน: การตรวจจับความผิดปกติ พบกันใหม่!**